<a href="https://colab.research.google.com/github/krzysztofnowakuz/AGH/blob/main/Decyclization_%26_Quick_heuristic_recovery_policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

data=pd.read_csv("testForm3.txt", sep=" ", header=None)
tabCol=[]
tabRow=[]

for col in data.columns:
    tabCol.append( "P" + str(col+1))
data.columns = tabCol

for row in data.index:
    tabRow.append("T" + str(row+1))
data.index=tabRow

incidenceMatrix = data.replace('x', -1)
incidenceMatrix = incidenceMatrix.astype(int)
initialMarking = incidenceMatrix.tail(1)
incidenceMatrix = incidenceMatrix[:-1]
numEdges, numVertices = incidenceMatrix.shape

#print (incidenceMatrix)
#print (initialMarking)


In [10]:
vertices = numVertices
vertices = list(range(1, vertices + 1))

edges = []

for row in incidenceMatrix.values:
    tails = [i+1 for i, x in enumerate(row) if x == -1]
    heads = [i+1 for i, x in enumerate(row) if x == 1]
    edges.extend([(tail, head) for tail in tails for head in heads])

print("Vertices =", vertices)
print("Edges =", edges)

Vertices = [1, 2, 3, 4, 5, 6, 7]
Edges = [(1, 2), (1, 3), (1, 4), (4, 5), (2, 6), (3, 6), (5, 6), (6, 7), (7, 1)]


In [11]:
head = [e[0] for e in edges]
tail = [e[1] for e in edges]

for e in edges:
    head[e[0]] += 1
    tail[e[1]] += 1

w = {}

for e in edges:
    w[e] = tail[e[1]] - head[e[1]] + head[e[0]] - tail[e[0]]
sorted_w = dict(sorted(w.items(), key=lambda item: item[1]))
E_prime = set()
Q = sorted_w

print(w)
print(Q)
print('Head', head)
print('Tail', tail)

{(1, 2): 3, (1, 3): 1, (1, 4): 4, (4, 5): -1, (2, 6): 0, (3, 6): 2, (5, 6): 0, (6, 7): -2, (7, 1): -1}
{(6, 7): -2, (4, 5): -1, (7, 1): -1, (2, 6): 0, (5, 6): 0, (1, 3): 1, (3, 6): 2, (1, 2): 3, (1, 4): 4}
Head [1, 4, 2, 5, 3, 4, 6, 7, 7]
Tail [2, 4, 5, 6, 7, 7, 9, 8, 1]


In [13]:
graph = {}
for tail, head in edges:
    if tail in graph:
        graph[tail].append(head)
    else:
        graph[tail] = [head]

dag_edges = []

def is_reachable(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return True
    if start not in graph:
        return False
    for node in graph[start]:
        if node not in path:
            newpath = is_reachable(graph, node, end, path)
            if newpath:
                return True
    return False

for tail, heads in graph.items():
    for head in heads:
        graph[tail].remove(head)
        if not is_reachable(graph, tail, head):
            dag_edges.append((tail, head))
        graph[tail].append(head)

print(dag_edges)

[(1, 2), (1, 4), (1, 4), (4, 5), (2, 6), (3, 6), (5, 6), (6, 7), (7, 1)]
